In [12]:
from config import DB_CONN_INFO,DB_URI
import pymysql
import pandas as pd
import sqlalchemy

In [5]:
connection = pymysql.connect(**DB_CONN_INFO)
engine = sqlalchemy.create_engine(DB_URI)

In [47]:
USER_STATS_SQL = """select l.id, 
    sum(l.sales_volume) as total_volume, 
    count(distinct l.uid) as num_lives, 
    sum(l.total_viewer) as total_viewer,
    avg(l.total_viewer) as avg_viewer,
    max(l.max_viewer) as max_viewer,
    g.goods_count as total_goods_count,
    g.total_sales_money as live_sales_money,
    g.total_sales_money / count(distinct l.uid) as avg_live_sales_money
from base_live l
left join (
    select id, 
        count(distinct goods_id) as goods_count, 
        sum(sales_price) as total_sales_money,
        sum(sales_count) as total_sales_count
    from base_live_goods
    group by id
) g on l.id = g.id
group by l.id
order by g.total_sales_count desc"""

TAG_SQL = '''select id, tag, sum(sales_count) as scount from base_live_goods
group by id, tag'''

In [93]:
def user_stats():
    connection = pymysql.connect(**DB_CONN_INFO)
    df = pd.read_sql_query(USER_STATS_SQL, connection)
    connection.close()
    return df

def user_tags():
    connection = pymysql.connect(**DB_CONN_INFO)
    df = pd.read_sql_query(TAG_SQL, connection)
    connection.close()
    
    blogger_tags = tag.sort_values(by = ['id','scount'],ascending = False)\
    .groupby('id')\
    .head(3)\
    .groupby('id')['tag'].apply(lambda x:list(x))

    blogger_tags = pd.DataFrame(blogger_tags)['tag'].apply(lambda x: [xx for xx in x if xx != '']).reset_index()
    blogger_tags['tag'] = blogger_tags['tag'].apply( lambda x: ','.join(x))
    return blogger_tags

def run_pipeline():
    engine = sqlalchemy.create_engine(DB_URI)
    df = user_stats().merge(user_tags(), on = 'id', how = 'left')
    df.to_sql('user_stats', con = engine, index = False, if_exists = 'replace')
    print('User stats completed')
    

In [89]:
df1 = user_tags()

In [94]:
df = user_stats().merge(user_tags(), on = 'id', how = 'left')
df.to_sql('user_stats', con = engine, index = False, if_exists = 'replace')

In [95]:
df.to_sql('user_stats', con = engine, index = False, if_exists = 'replace')

In [20]:
tag_sql = '''select id, tag, sum(sales_count) as scount from base_live_goods
group by id, tag'''

tag = pd.read_sql_query(tag_sql, connection)

In [35]:
blogger_tags = tag.sort_values(by = ['id','scount'],ascending = False)\
.groupby('id')\
.head(3)\
.groupby('id')['tag'].apply(lambda x:list(x))

blogger_tags = pd.DataFrame(blogger_tags)['tag'].apply(lambda x: [xx for xx in x if xx != '']).reset_index()

In [19]:
说明： 所有有直播的用户中，按照直播销售总额倒序。





主播粒度： id, name, fans, douyin



聚合粒度：

live_sales_count: 所有直播加起来的总销量

num_lives: 直播数量

total_viewer: 所有直播加来的观看量

avg_viewer: 平均单场直播的观看量

max_viewer: 所有直播的所有时刻的最大观看量

total_goods_count: 所有直播中带过的货品的总量

live_sales_money: 所有直播加起来总带货收入

avg_lives_sales_money: 平均单场直播收入

goods_tag_list: 按照降序排列的品类前5个

SyntaxError: invalid character in identifier (<ipython-input-19-4bc0734410f0>, line 1)